<a href="https://colab.research.google.com/github/marcosvnespolo/Previsao_Vendas/blob/main/Previs%C3%A3o_de_Vendas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Importando as Bibliotecas Necessárias

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
import lightgbm as lgb


# 2. Carregando os Datasets Utilizados no Problema

In [ ]:
features = pd.read_csv('/content/drive/MyDrive/Datasets/Previsão de Vendas/features.csv')
stores = pd.read_csv('/content/drive/MyDrive/Datasets/Previsão de Vendas/stores.csv')
test = pd.read_csv('/content/drive/MyDrive/Datasets/Previsão de Vendas/test.csv')
train = pd.read_csv('/content/drive/MyDrive/Datasets/Previsão de Vendas/train.csv')

# 3. Visualizando os Datasets

In [ ]:
display(features.head())
print()
print()
display(stores.head())
print()
print()
display(test.head())
print()
print()
display(train.head())

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


,Store,Dept,Date,IsHoliday
0,1,1,2012-11-02,False
1,1,1,2012-11-09,False
2,1,1,2012-11-16,False
3,1,1,2012-11-23,True
4,1,1,2012-11-30,False


,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False


# 4. Juntando os Datasets

É importante relacionar os datasets porque todos eles possuem imformações relevantes que se complementam entre sí

In [ ]:
train = pd.merge(train, features, on=['Store', 'Date', 'IsHoliday'], how='left')
train = pd.merge(train, stores, on=['Store'], how='left')
test = pd.merge(test, features, on=['Store', 'Date', 'IsHoliday'], how='left')
test = pd.merge(test, stores, on=['Store'], how='left')

# 5. Limpeza e Pré-Processamento dos dados

Nesta etatpa, irei preencher os valores ausentes e converter as variáveis categóricas em numéricas usando o LabelEncoder.

In [ ]:
train.fillna(-999, inplace=True)
test.fillna(-999, inplace=True)
train['Date'] = pd.to_datetime(train['Date'])
test['Date'] = pd.to_datetime(test['Date'])
le = LabelEncoder()
train['Type'] = le.fit_transform(train['Type'])
test['Type'] = le.transform(test['Type'])
train['IsHoliday'] = train['IsHoliday'].astype(int)
test['IsHoliday'] = test['IsHoliday'].astype(int)

# 6. Feature Engineering
Nesta etapa, o objetivo é extrair informações adicionais da **variáveis** de data, como ano, mês, dia e semana do ano.

In [7]:
train['Year'] = train['Date'].dt.year
train['Month'] = train['Date'].dt.month
train['Day'] = train['Date'].dt.day
train['WeekOfYear'] = train['Date'].dt.weekofyear
test['Year'] = test['Date'].dt.year
test['Month'] = test['Date'].dt.month
test['Day'] = test['Date'].dt.day
test['WeekOfYear'] = test['Date'].dt.weekofyear


<ipython-input-7-302956a1e565>:4: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  train['WeekOfYear'] = train['Date'].dt.weekofyear
<ipython-input-7-302956a1e565>:8: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  test['WeekOfYear'] = test['Date'].dt.weekofyear


# 7. Removendo a Coluna Date
As informações mais relevantes da coluna date já foram extraídas na etapa anterior, então, podemos removê-la sem maiores problemas

In [8]:
train.drop('Date', axis=1, inplace=True)
test.drop('Date', axis=1, inplace=True)

In [10]:
display(train.head())

,Store,Dept,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size,Year,Month,Day,WeekOfYear
0,1,1,24924.50,0,42.31,2.572,-999.0,-999.0,-999.0,-999.0,-999.0,211.096358,8.106,0,151315,2010,2,5,5
1,1,1,46039.49,1,38.51,2.548,-999.0,-999.0,-999.0,-999.0,-999.0,211.242170,8.106,0,151315,2010,2,12,6
2,1,1,41595.55,0,39.93,2.514,-999.0,-999.0,-999.0,-999.0,-999.0,211.289143,8.106,0,151315,2010,2,19,7
3,1,1,19403.54,0,46.63,2.561,-999.0,-999.0,-999.0,-999.0,-999.0,211.319643,8.106,0,151315,2010,2,26,8
4,1,1,21827.90,0,46.50,2.625,-999.0,-999.0,-999.0,-999.0,-999.0,211.350143,8.106,0,151315,2010,3,5,9


In [11]:
display(test.head())

,Store,Dept,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size,Year,Month,Day,WeekOfYear
0,1,1,0,55.32,3.386,6766.44,5147.70,50.82,3639.90,2737.42,223.462779,6.573,0,151315,2012,11,2,44
1,1,1,0,61.24,3.314,11421.32,3370.89,40.28,4646.79,6154.16,223.481307,6.573,0,151315,2012,11,9,45
2,1,1,0,52.92,3.252,9696.28,292.10,103.78,1133.15,6612.69,223.512911,6.573,0,151315,2012,11,16,46
3,1,1,1,56.23,3.211,883.59,4.17,74910.32,209.91,303.32,223.561947,6.573,0,151315,2012,11,23,47
4,1,1,0,52.34,3.207,2460.03,-999.00,3838.35,150.57,6966.34,223.610984,6.573,0,151315,2012,11,30,48


# 8. Criando o Conjunto de Treino e Teste
O objetivo desta etapa é dividir o conjunto de dados de treinamento em um conjunto de treinamento e validação.

In [12]:
X_train, X_val, y_train, y_val = train_test_split(train.drop('Weekly_Sales', axis=1), train['Weekly_Sales'], test_size=0.2, random_state=42)
